In [1]:
%run nb-env.ipynb

[2021-12-23T20:51:05.748897+08:00] Enviroment loaded. Working Dir: /Users/tzhu/work/lab/neo_world/notebooks
Today is 2021-12-23, Working from 2020-12-08 to 2021-12-23


In [2]:
# check if adj_factor is changed
genesis_date = '2010-01-04'
print(f'today_date: {today_date}, end_date: {end_date}, yesterday_date: {yesterday_date}')

today_date: 2021-12-23, end_date: 2021-12-23, yesterday_date: 2021-12-22


In [5]:
def get_xd_stocks(today_date, yesterday_date):
    '''
    Get Stocks XDed at given day
    '''
    adj = load_table(start_date=yesterday_date, end_date=today_date, model=AdjFactor)
    adj_today = adj.xs(level='trade_date', key=today_date, drop_level=True)
    adj_yesterday = adj.xs(level='trade_date', key=yesterday_date, drop_level=True)
    adj_diff = adj_yesterday[(adj_yesterday.adj_factor - adj_today.adj_factor) != 0]
    xd_stocks = adj_diff.index.to_list()
    return xd_stocks


def get_adj_data(start_date, end_date, xd_stocks):
    '''
    Calculate adj price/vol from start_date to end_date
    '''
    prices = load_stock_prices(start_date=start_date, end_date=end_date, ts_codes=xd_stocks)
    adj_prices = gen_adj_price(prices, replace=True)
    return adj_prices


def upsert_df(df, table, engine, cols=None):
    '''
    Fast insert with temp_table
    '''
    if cols is None:
        cols = df.columns.to_list()
    df[cols + ['ts_code', 'trade_date']].to_sql('temp_table', engine, if_exists='replace', index=False)
    set_str = ', '.join(list(map(lambda col: f'{col} = t.{col}', cols)))    
    sql = f"""
        UPDATE {table} AS f
        SET {set_str}
        FROM temp_table AS t
        WHERE f.trade_date = t.trade_date and f.ts_code = t.ts_code
    """
    with engine.begin() as conn:     # TRANSACTION
        conn.execute(sql)

In [6]:
xd_stocks = get_xd_stocks(end_date, yesterday_date)
stock_list = StockFilter(end_date=today_date).get_list()
print(f'Total stocks: {len(stock_list)}, xd stocks: {len(xd_stocks)}')

Total stocks: 4588, xd stocks: 1


In [7]:
%%time

# 非 xd stock，只更新今日 prices
price = load_stock_prices(start_date=end_date, end_date=end_date, ts_codes=subtract(stock_list,xd_stocks))
price.rename(columns={'vol':'adj_vol'}, inplace=True)
price = price[['open', 'close', 'high', 'low', 'adj_vol']].reset_index()
insert_df(price, tablename='daily_tick')
print(f'Total stocks updated: {len(price)}')

Total stocks updated: 4581
CPU times: user 20.7 s, sys: 1.18 s, total: 21.9 s
Wall time: 37.9 s


In [8]:
%%time

# update xd stocks, from genesis_date
if len(xd_stocks) > 0:
    adj_prices = get_adj_data(genesis_date, today_date, xd_stocks=xd_stocks)
    adj_df = adj_prices[['open', 'close', 'high', 'low', 'adj_vol']].reset_index()
    insert_df(adj_df, tablename='daily_tick')

CPU times: user 12.8 s, sys: 780 ms, total: 13.6 s
Wall time: 24.7 s


In [9]:
adj_df

,ts_code,trade_date,open,close,high,low,adj_vol
0,601555.SH,2011-12-12,6.36,5.80,6.45,5.70,2911424.92
1,601555.SH,2011-12-13,5.87,5.93,6.12,5.74,1890103.81
2,601555.SH,2011-12-14,5.88,6.00,6.05,5.59,1332425.97
3,601555.SH,2011-12-15,5.86,5.78,6.28,5.75,1733432.44
4,601555.SH,2011-12-16,5.68,5.97,6.00,5.61,1386882.94
...,...,...,...,...,...,...,...
2415,601555.SH,2021-12-09,8.84,9.00,9.17,8.79,980649.84
2416,601555.SH,2021-12-10,8.59,8.51,8.67,8.37,2093040.27
2417,601555.SH,2021-12-13,8.49,8.47,8.65,8.45,1596812.39
2418,601555.SH,2021-12-14,8.38,8.48,8.54,8.35,1071443.47


### calc history and save

In [52]:


#     # pct_chg
#    pct_chg_p1 = Column(Float(precision=4, asdecimal=True))   # 2日累积涨幅
#    pct_chg_p2 = Column(Float(precision=4, asdecimal=True))   # 3日累积涨幅
#    pct_chg_p4 = Column(Float(precision=4, asdecimal=True))   # 5日累积涨幅
#    pct_chg_p9 = Column(Float(precision=4, asdecimal=True))   # 10日累积涨幅

# next2_pct_chg_p1 = Column(Float(precision=4, asdecimal=True))   # 未来2日累积涨幅
# next3_pct_chg_p2 = Column(Float(precision=4, asdecimal=True))   # 未来3日累积涨幅
# next5_pct_chg_p4 = Column(Float(precision=4, asdecimal=True))   # 未来5日累积涨幅
# next10_pct_chg_p9 = Column(Float(precision=4, asdecimal=True))   # 未来10日累积涨幅

# open pct
# open_pct = Column(Float(precision=4, asdecimal=True))   # 未来1日开盘涨幅
# close_v_open =  Column(Float(precision=4, asdecimal=True))   # 收盘相对开盘涨幅
# next_open_pct = Column(Float(precision=4, asdecimal=True))   # 未来1日开盘涨幅
# next_cvo = Column(Float(precision=4, asdecimal=True))   # 未来1日收盘相对开盘涨幅

#     # price, max/min
#     max_high_p59 = Column(Float(precision=2, asdecimal=True))   # 过去 60 天最高价
#     min_low_p59 = Column(Float(precision=2, asdecimal=True))   # 过去 60 天最高价

#     # ma close/vol/amount

#     # trf 
#     pre_trf   = Column(Float(precision=4, asdecimal=True))    # 昨日实际换手率
#     ma_trf_3  = Column(Float(precision=4, asdecimal=True))
#     ma_trf_5  = Column(Float(precision=4, asdecimal=True))
#     ma_trf_10 = Column(Float(precision=4, asdecimal=True))

#     # 量比 (in 5 days)
#     vol_ratio = Column(Float(precision=4, asdecimal=True))
#     # 量比 (in 20 days)
#     vol_ratio_long = Column(Float(precision=4, asdecimal=True))

#     # vol_type
#     vol_type = Column(String(4))
#     bar_type = Column(String(4))

#     # upstops
#     up_type = Column(String(4))
#     conseq_up_num = Column(Integer)
#     upstops_p2 = Column(Integer)    # 过去 3 天涨停次数
#     upstops_p4 = Column(Integer)    # 过去 5 天涨停次数
#     upstops_p9 = Column(Integer)    # 过去 10 天涨停次数
#     upstops_p19 = Column(Integer)   # 过去 20 天涨停次数

for stock in stock_list:
    stock_df = load_table(start_date=genesis_date, end_date=end_date, model=DailyTick, ts_codes=[stock])
    price_df = load_table(start_date=genesis_date, end_date=end_date, model=Price, ts_codes=[stock])
    up_df = load_table(start_date=genesis_date, end_date=end_date, model=UpStop, ts_codes=[stock])
    stock_df.loc[:,'amount']=price_df.amount
    stock_df.loc[:,'limit']=price_df.limit
    # pre_close
    for t_minus_days in [0, 1, 2, 4, 9]:
        stock_df.loc[:,f'pre{t_minus_days+1}_close'] = stock_df.groupby('ts_code').close.shift(t_minus_days+1)
        stock_df.loc[:,f'pct_chg_p{t_minus_days}'] = round((stock_df.close/stock_df[f'pre{t_minus_days+1}_close']-1)*100,4)
    stock_df.rename(columns={'pct_chg_p0':'pct_chg', 'pre1_close':'pre_close', 'adj_vol':'vol'}, inplace=True)
    
    # open_pct
    stock_df.loc[:,'open_pct'] = round((stock_df.open/stock_df.pre_close-1)*100, 4)
    stock_df.loc[:, 'close_v_open'] = stock_df.pct_chg - stock_df.open_pct
    # next_open_pct = Column(Float(precision=4, asdecimal=True))   # 未来1日开盘涨幅
    # next_cvo = Column(Float(precision=4, asdecimal=True))   # 未来1日收盘相对开盘涨幅
    
    stock_df = gen_ma(stock_df, mavgs=[5, 10, 20, 30, 60, 120, 250], col='close', add_shift=0, single_index=False)
    stock_df = gen_ma(stock_df, mavgs=[3, 5, 10, 20, 30, 60, 120, 250], col='vol', add_shift=0, single_index=False)
    stock_df = gen_ma(stock_df, mavgs=[3, 5, 10, 20, 30, 60, 120, 250], col='amount', add_shift=0, single_index=False)
    break

In [53]:
stock_df[['pre_close', 'open', 'open_pct', 'close', 'close_v_open', 'pct_chg', 'pct_chg_p1', 'pct_chg_p4', 'pct_chg_p9', 'ma_close_5', 'ma_close_10', 'ma_close_250']]

pre_close   open  open_pct  close  close_v_open  \
ts_code   trade_date                                                    
000001.SZ 2010-01-04        NaN   7.87       NaN   7.61           NaN   
          2010-01-05       7.61   7.62    0.1314   7.48       -1.8397   
          2010-01-06       7.48   7.46   -0.2674   7.35       -1.4706   
          2010-01-07       7.35   7.35    0.0000   7.27       -1.0884   
          2010-01-08       7.27   7.22   -0.6878   7.25        0.4127   
...                         ...    ...       ...    ...           ...   
          2021-12-16      17.55  17.55    0.0000  17.72        0.9687   
          2021-12-17      17.72  17.70   -0.1129  17.57       -0.7336   
          2021-12-20      17.57  17.56   -0.0569  17.52       -0.2277   
          2021-12-21      17.52  17.49   -0.1712  17.59        0.5707   
          2021-12-22      17.59  17.62    0.1706  17.39       -1.3076   

                      pct_chg  pct_chg_p1  pct_chg_p4  pct_chg_p9  ma_close_5  \
ts_code   trade_date                                                            
000001.SZ 2010-01-04      NaN         NaN         NaN         NaN         NaN   
          2010-01-05  -1.7083         NaN         NaN         NaN         NaN   
          2010-01-06  -1.7380     -3.4166         NaN         NaN         NaN   
          2010-01-07  -1.0884     -2.8075         NaN         NaN         NaN   
          2010-01-08  -0.2751     -1.3605         NaN         NaN       7.392   
...                       ...         ...         ...         ...         ...   
          2021-12-16   0.9687      0.7964     -5.2406      0.7391      17.900   
          2021-12-17  -0.8465      0.1140     -4.4070     -0.4533      17.738   
          2021-12-20  -0.2846     -1.1287     -4.1051     -3.2044      17.588   
          2021-12-21   0.3995      0.1138      0.0569     -4.7129      17.590   
          2021-12-22  -1.1370     -0.7420     -0.9117     -5.8473      17.558   

                      ma_close_10  ma_close_250  
ts_code   trade_date                             
000001.SZ 2010-01-04          NaN           NaN  
          2010-01-05          NaN           NaN  
          2010-01-06          NaN           NaN  
          2010-01-07          NaN           NaN  
          2010-01-08          NaN           NaN  
...                           ...           ...  
          2021-12-16       18.088      20.56572  
          2021-12-17       18.080      20.56176  
          2021-12-20       18.022      20.55760  
          2021-12-21       17.935      20.55396  
          2021-12-22       17.827      20.55000  

[2840 rows x 12 columns]

In [54]:
stock_df[['ma_vol_5', 'ma_vol_10', 'ma_vol_250', 'ma_amount_5', 'ma_amount_10', 'ma_amount_250']]

ma_vol_5    ma_vol_10    ma_vol_250  ma_amount_5  \
ts_code   trade_date                                                        
000001.SZ 2010-01-04          NaN          NaN           NaN          NaN   
          2010-01-05          NaN          NaN           NaN          NaN   
          2010-01-06          NaN          NaN           NaN          NaN   
          2010-01-07          NaN          NaN           NaN          NaN   
          2010-01-08  1156082.040          NaN           NaN    854602.78   
...                           ...          ...           ...          ...   
          2021-12-16  1271012.220  1326089.775  979926.10784   2280268.18   
          2021-12-17  1193635.140  1315566.770  979434.36664   2128027.98   
          2021-12-20  1080024.140  1175647.770  979689.92720   1904273.06   
          2021-12-21   871329.172  1103376.786  980814.28244   1532020.94   
          2021-12-22   807789.242  1103041.441  981142.90044   1418411.26   

                      ma_amount_10  ma_amount_250  
ts_code   trade_date                               
000001.SZ 2010-01-04           NaN            NaN  
          2010-01-05           NaN            NaN  
          2010-01-06           NaN            NaN  
          2010-01-07           NaN            NaN  
          2010-01-08           NaN            NaN  
...                            ...            ...  
          2021-12-16    2404542.46   1.990423e+06  
          2021-12-17    2386592.51   1.989290e+06  
          2021-12-20    2127856.91   1.989416e+06  
          2021-12-21    1987149.57   1.991184e+06  
          2021-12-22    1977457.73   1.991422e+06  

[2840 rows x 6 columns]

### calc new incrementally

In [20]:
adj_prices.loc[:, 'open_pct'] = round((adj_prices.open - adj_prices.pre_close)/adj_prices.pre_close * 100, 4)
adj_prices.loc[:, 'close_v_open'] = adj_prices.pct_chg - adj_prices.open_pct

In [21]:
adj_prices.loc['688601.SH'][['vol', 'adj_vol', 'turnover_rate', 'pre_close', 'open', 'close', 'open_pct', 'pct_chg', 'close_v_open', 'adj_factor']]

,vol,adj_vol,turnover_rate,pre_close,open,close,open_pct,pct_chg,close_v_open,adj_factor
trade_date,,,,,,,,,,
2021-06-28,106010.79,106254.61,81.3799,36.40,159.63,155.64,338.5440,327.6316,-10.9124,1.0000
2021-06-29,66265.28,66417.69,50.8690,155.64,155.85,164.62,0.1349,5.7692,5.6343,1.0000
2021-06-30,65886.98,66038.52,50.5786,164.62,168.49,183.68,2.3509,11.5758,9.2249,1.0000
2021-07-01,51580.26,51698.89,39.5959,183.68,176.71,161.43,-3.7946,-12.1130,-8.3184,1.0000
2021-07-02,28597.30,28663.07,21.9529,161.43,161.73,164.93,0.1858,2.1693,1.9835,1.0000
...,...,...,...,...,...,...,...,...,...,...
2021-12-14,4384.48,4394.56,3.3658,166.62,167.67,165.51,0.6302,-0.6647,-1.2949,1.0000
2021-12-15,5557.67,5570.45,4.2664,165.51,166.60,159.23,0.6586,-3.7917,-4.4503,1.0000
2021-12-16,4341.14,4351.12,3.3325,159.23,159.21,158.32,-0.0126,-0.5764,-0.5638,1.0000


In [28]:


# test cols is None
op_df = adj_prices[['open_pct', 'close_v_open']].reset_index()
upsert_df(op_df, 'daily_tick', engine)

In [ ]:
# test cols param
adj_prices.loc[:, 'next_open_pct'] = adj_prices.groupby('ts_code').open_pct.shift(-1)
upsert_df(adj_prices.reset_index(), 'daily_tick', engine, cols=['next_open_pct'])

,ts_code,trade_date,adj_factor
4496,688601.SH,20211217,1.0


In [50]:
df2 = pro.adj_factor(trade_date=pdl.parse(today_date).strftime("%Y%m%d"))
df2[df2.ts_code=='688601.SH']

,ts_code,trade_date,adj_factor
4497,688601.SH,20211220,1.0023


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
4496,688601.SH,20211217,158.88,158.88,150.65,151.37,158.68,-7.31,-4.6068,6544.4,99953.03


In [52]:
(151.37/158.68-1)

-0.04606755734812207

In [48]:
151.02/158.33-1

-0.04616939303985346